## 21D008 Probabilistic Inference - Project: Elliptical Slice Sampling in Classification

In [2]:
from scipy.stats import multivariate_normal
from EllipticalSliceSampler import EllipticalSampler
from utils import (
    plot_sampler,
    plot_multiple
)

import numpy as np

In [4]:
prior_µ = np.array([1,2])